In [ ]:
print("OK")

In [2]:
import os
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import CTransformers
# from langchain.retrievers import BaseRetriever
from typing import List, Dict

In [3]:

# import os
# from pinecone import Pinecone
# PINECONE_API_KEY="25acba92-7698-44b7-91cb-8bb4f7e0d30c"
# PINECONE_API_ENV="Serverless"


# initialize connection to pinecone (get API key at app.pinecone.io)
# api_key = os.environ.get('PINECONE_API_KEY')

# configure client
# pc = Pinecone(api_key)

In [4]:
def load_pdf(data):
    loader=DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents


In [5]:
extracted_data = load_pdf("data/")

In [6]:
# extracted_data

In [7]:
#Text Chunks
def text_splits(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)

    return text_chunks

In [8]:
text_chunks=text_splits(extracted_data)
print("Length of my chunk",len(text_chunks))

Length of my chunk 7020


In [9]:
#text_chunks

In [10]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    return embeddings

In [11]:
embeddings = download_hugging_face_embeddings()
embeddings_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [12]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [13]:
query_result = embeddings_model.encode("Hello world", convert_to_tensor=False)
print("Length", len(query_result))

Length 384


In [14]:
# query_result

In [15]:
texts = [t.page_content for t in text_chunks]
embeddings = embeddings_model.encode(texts, convert_to_tensor=False)

# Convert embeddings to numpy array
embeddings_array = np.array(embeddings).astype('float32')

# Create a FAISS index
d = embeddings_array.shape[1]  # dimension of embeddings
index = faiss.IndexFlatL2(d)  # L2 distance index
index.add(embeddings_array)  # add vectors to the index

print("Data successfully indexed!")

# Function to perform similarity search
def search(query, k=5):
    query_embedding = embeddings_model.encode([query], convert_to_tensor=False)
    query_embedding = np.array(query_embedding).astype('float32')
    distances, indices = index.search(query_embedding, k)  # search
    results = [(texts[i], distances[0][j]) for j, i in enumerate(indices[0])]
    return results

# Example search
query = "Example query text."
results = search(query, k=3)
for result in results:
    print(f"Text: {result[0]}, Distance: {result[1]}")


Data successfully indexed!
Text: ability to communicate and record events with text)., Distance: 1.378244161605835
Text: identify whichKEY TERMS, Distance: 1.4424941539764404
Text: individuals are highlighted as sidebar biographies thataccompany the main topical essays. Articles follow astandardized format that provides information at aglance. Rubrics include:
Disorders/Conditions Tests/Treatments
Definition Definition
Description Purpose
Causes and symptoms Precautions
Diagnosis DescriptionTreatment Preparation
Alternative treatment Aftercare
Prognosis Risks
Prevention Normal/Abnormal results
Resources ResourcesKey terms Key terms, Distance: 1.4636788368225098


In [16]:
query = "What are Allergies."
results = search(query, k=3)
for result in results:
    print(f"Text: {result[0]}, Distance: {result[1]}")

Text: GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies
Allergic rhinitis is commonly triggered by
exposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.
The presence of an allergen causes the
body's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.
IgE molecules attach to mast
cells, which contain histamine.HistaminePollen grains
Lymphocyte
FIRST EXPOSURE, Distance: 0.6048821210861206
Text: itchy, scratchy nose, eyes, and throat common in aller-gic rhinitis.
The number of possible airborne allergens is enor-, Distance: 0.6296976208686829
Text: allergens are the following:
• plant pollens
• animal fur and dander
• body parts from house mites (microscopic creatures
found in all houses)
• house dust• mold spores• cigarette smoke• solvents• cleaners
Common food allergens include the following:
• nuts, especially peanuts, walnuts, and brazil nuts
•

In [17]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.5})

In [18]:
# Define additional chain type kwargs if needed
chain_type_kwargs = {}

In [19]:
# # Initialize the RetrievalQA chain
# qa = RetrievalQA.from_chain_type(
#     llm=llm, 
#     chain_type="stuff", 
#     retriever=None,  # No retriever needed
#     return_source_documents=True, 
#     chain_type_kwargs=chain_type_kwargs
# )

In [20]:
# # Perform a query
# qa_query = "What are Allergies?"
# qa_result = qa({"query": qa_query})

# # Print the results
# print("QA Result:", qa_result["result"])
# print("Source Documents:", qa_result["source_documents"])


In [21]:
#IMP 
# def query_faiss(query, k=5):
#     query_embedding = embeddings_model.encode([query], convert_to_tensor=False)
#     query_embedding = np.array(query_embedding).astype('float32')
#     D, I = index.search(query_embedding, k)  # search the top k nearest neighbors
#     return I[0]  
# def generate_answer(question, documents):
#     context = "\n".join([doc.page_content for doc in documents])
#     prompt = f"Context: {context}\n\nQuestion: {question}\nAnswer:"
#     response = llm(prompt)
#     return response
# # Perform a query
# qa_query = "What are Allergies?"
# top_k_indices = query_faiss(qa_query, k=5)

# # Retrieve the most relevant document chunks
# retrieved_docs = [text_chunks[i] for i in top_k_indices]

# # Generate the answer
# qa_result = generate_answer(qa_query, retrieved_docs)

# # Print the results
# print("QA Result:", qa_result)
# print("Source Documents:", [doc.page_content for doc in retrieved_docs])

In [22]:
query = "What is Jaundice."
results = search(query, k=3)
for result in results:
    print(f"Text: {result[0]}, Distance: {result[1]}")

Text: • Thorotrast. This is a chemical that was previously
injected intravenously during certain types of x rays. Itis not in use anymore. Exposure to Thorotrast has beenimplicated in the development of cancer of the liver aswell as the bile ducts.
Symptoms
Jaundice is the first symptom in 90% of patients., Distance: 0.8480944633483887
Text: stools will probably be clay-colored. At this point, thecondition will come to the attention of a physician. If not,the child’s abdomen will begin to swell, and the infantwill get progressively more ill. Nearly all untreated chil-dren will die of liver failure within two years.
Diagnosis
The persistence of jaundice beyond the second week, Distance: 0.9045239686965942
Text: the invasion of connective tissue and the degenera-tion of proper functioning—jaundice is often anaccompanying symptom. Causes of cirrhosis includealcoholism, metabolic diseases, syphilis and conges-tive heart disease.
Common bile duct —The branching passage
through which bile—a 

In [23]:
def summarize_text(text, max_length=100):
    # A simple summarization method (this is a placeholder)
    # For better results, consider using an NLP library like Hugging Face's transformers for summarization
    sentences = text.split('. ')
    summarized = ' '.join(sentences[:2])  # Taking the first two sentences as an example
    return summarized[:max_length]  # Truncate to max_length
query = "What is acene."
results = search(query, k=3)

# Summarize and print the results
for result in results:
    text, distance = result[0], result[1]
    summarized_text = summarize_text(text)
    # print(f"Text: {summarized_text}, Distance: {distance}")
    print(summarized_text)
def summarize_text(text, max_length=100):
    # A simple summarization method
    sentences = text.split('. ')
    summarized = ' '.join(sentences[:2])  # Taking the first two sentences as an example
    return summarized[:max_length]  # Truncate to max_length

# def search(query, k=3):
#     # This function should perform the actual search and return results
#     # For this example, it's a placeholder and should be replaced with actual search logic
#     return [
#         ("GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies Allergic rhinitis is commonly triggered by exposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen. The presence of an allergen causes the body's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE. IgE molecules attach to mast cells, which contain histamine.HistaminePollen grains Lymphocyte FIRST EXPOSURE", 0.6048821210861206),
#         ("itchy, scratchy nose, eyes, and throat common in aller-gic rhinitis. The number of possible airborne allergens is enor-", 0.6296976208686829),
#         ("allergens are the following: • plant pollens • animal fur and dander • body parts from house mites (microscopic creatures found in all houses) • house dust• mold spores• cigarette smoke• solvents• cleaners Common food allergens include the following: • nuts, especially peanuts, walnuts, and brazil nuts • fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives The following types of drugs commonly cause aller- gic reactions: • penicillin or other antibiotics", 0.6369765996932983)
#     ]

# query = "What are Allergies."
# results = search(query, k=3)

# # Summarize and print the results
# for result in results:
#     text, distance = result[0], result[1]
#     summarized_text = summarize_text(text)
#     print(f"Text: {summarized_text}, Distance: {distance}")


animal, usually delivered through a bite or a sting.ACE is to help regulate arterial pressure by con
called ACE inhibitors) are medicines that block the con-version of the chemical angiotensin I to a s
normally have very high ACE levels Decreased levels maybe seen in the condition of excess fat in the


In [25]:
query = "What is achalasia"
results = search(query, k=3)
for result in results:
    print(f"Text: {result[0]}, Distance: {result[1]}")

Text: Check with a physician or pharmacist before combining
acetaminophen with any other prescription or nonpre-scription (over-the-counter) medicine.
Nancy Ross-Flanigan
Acetylsalicylic acid seeAspirin
Achalasia
Definition
Achalasia is a disorder of the esophagus that pre-
vents normal swallowing.
Description
Achalasia affects the esophagus, the tube that carries, Distance: 0.6180734634399414
Text: cells that normally signal the brain to relax theesophageal sphincter. The ultimate cause of this degener-ation is unknown. Autoimmune disease or hidden infec-tion is suspected.
Symptoms
Dysphagia, or difficulty swallowing, is the most com-
mon symptom of achalasia. The person with achalasia usu-ally has trouble swallowing both liquid and solid foods,often feeling that food “gets stuck” on the way down. Theperson has chest pain that is often mistaken for angina, Distance: 0.7626405954360962
Text: esophageal contraction called peristalsis. Peristalsis pushesfood along the esophagus. Normally

In [32]:
query = '''why disesase i have i am using these medication Tab.PCM 500mg fever, headache , body pains vachinappudu
 Montek LC 9pm for 3 days
 Strepcils(throat losengen) irritation vachinappuadalla 
 Limce 500mg 8 am 1pm 8 pm. Saparinchu oka 2 days'''
results = search(query, k=3)
for result in results:
    print(f"Text: {result[0]}")

Text: serious blood disorders. Anyone taking this medicineshould have regular blood counts and should check witha physician if any of the following symptoms occur:
• joint or muscle pain
• muscle weakness
• pain in the chest or abdomen
• tremors
• wheezing
• cough
• palpitations
• rash, sores, or pain in the mouth
• sore throat
• fever and chills
• loss of appetite
•diarrhea
• dark urine
• yellow skin or eyes
• unusual bleeding or bruising• dizziness
•hallucinations
• depression
Special conditions
Text: dizziness, fatigue, seizuresWellbutrin (bupropion hydrochloride) Agitation, dry mouth, headache, nausea, rashZoloft (sertraline) Diarrhea, fainting, gas, headache, nervousnessGEM - 0001 to 0432 - A  10/22/03 1:42 PM  Page 257
Text: Phenergan (promethazine hydrochloride) Dizziness, dry mouth, nausea and vomiting, rashReglan (metoclopramide hydrochloride) Fatigue, drowsiness, restlessnessTigan (trimethobenzamide hydrochloride) Blurred vision, diarrhea, cramps, headacheZofan (ondansetron h